In [ ]:
import os
import numpy as np
from lsst.utils import getPackageDir
from lsst.sims.photUtils import Bandpass, Sed

nSpectra = 20

sedDir = os.path.join(getPackageDir('sims_sed_library'), 'starSED', 'kurucz')
bandpassDir = os.path.join(getPackageDir('throughputs'), 'baseline')

np.random.seed(42)

# create a random list of SED file names drawn from sedDir
sedFullNameList = np.array(os.listdir(sedDir))
sedDexes = np.unique(np.random.random_integers(0, len(sedFullNameList), nSpectra))
sedNameList = sedFullNameList[sedDexes]

# create a random list of normalizing magnitudes
magNormList = np.random.random_sample(nSpectra)*10.0 + 18.0

# read in the r bandpass
rBandpass = Bandpass()
rBandpass.readThroughput(os.path.join(bandpassDir, 'total_r.dat'))

sedList = []
for name, norm in zip(sedNameList, magNormList):

    fullName = os.path.join(sedDir, name)

    # instantiate and read in the SED
    spectrum = Sed()
    spectrum.readSED_flambda(fullName)

    # normalize the SED to have the specified magnitude in the r band
    ff = spectrum.calcFluxNorm(norm, rBandpass)
    spectrum.multiplyFluxNorm(ff)
    sedList.append(spectrum)

In [ ]:
total_i_bandpass = Bandpass()
total_i_bandpass.readThroughput(os.path.join(bandpassDir, 'total_i.dat'))

In [ ]:
test_i_bandpass = Bandpass()
componentList = ['detector.dat', 'm1.dat', 'm2.dat', 'm3.dat', 
                 'lens1.dat', 'lens2.dat', 'lens3.dat', 'filter_i.dat', 'atmos.dat']
test_i_bandpass.readThroughputList(componentList, rootDir=bandpassDir)

np.testing.assert_array_equal(test_i_bandpass.wavelen, total_i_bandpass.wavelen)
np.testing.assert_allclose(test_i_bandpass.sb,
                           total_i_bandpass.sb,
                           rtol=10)

In [ ]:
hardware_i_bandpass = Bandpass()
componentList = ['detector.dat', 'm1.dat', 'm2.dat', 'm3.dat', 
                 'lens1.dat', 'lens2.dat', 'lens3.dat', 'filter_i.dat']
hardware_i_bandpass.readThroughputList(componentList, rootDir=bandpassDir)

In [ ]:
skySed = Sed()
skySed.readSED_flambda(os.path.join(bandpassDir, 'darksky.dat'))

In [ ]:
from lsst.sims.photUtils import PhotometricParameters

photParams = PhotometricParameters()

In [ ]:
help(PhotometricParameters.__init__)

In [ ]:
from lsst.sims.photUtils import LSSTdefaults

defaults = LSSTdefaults()

print help(defaults.FWHMeff)
print '\n'
print help(defaults.effwavelen)
print '\n'
print help(defaults.m5)
print '\n'
print help(defaults.gamma)

In [ ]:
import lsst.sims.photUtils.SignalToNoise as snr
m5 = snr.calcM5(skySed, total_i_bandpass, hardware_i_bandpass, photParams, FWHMeff=0.8)
print m5

In [ ]:
snr_sed_list = []
for spectrum in sedList:
    val = snr.calcSNR_sed(spectrum, total_i_bandpass, skySed, hardware_i_bandpass,
                         photParams, 0.8)
    snr_sed_list.append(val)

snr_sed_list = np.array(snr_sed_list)

In [ ]:
magList = []
for spectrum in sedList:
    mm = spectrum.calcMag(total_i_bandpass)
    magList.append(mm)
magList = np.array(magList)

snr_m5_list, gamma = snr.calcSNR_m5(magList, total_i_bandpass, m5, photParams)

In [ ]:
print 'maximum deviation in snr: ',np.abs(snr_sed_list - snr_m5_list).max()

print 'value of snr at maximum deviation: ', \
       snr_sed_list[np.argmax(np.abs(snr_sed_list - snr_m5_list))]

In [ ]:
mag_error_sed_list = []
for spectrum in sedList:
    sigma = snr.calcMagError_sed(spectrum, total_i_bandpass, skySed,
                                hardware_i_bandpass, photParams,
                                0.8)
    mag_error_sed_list.append(sigma)

mag_error_sed_list = np.array(mag_error_sed_list)

In [ ]:
mag_error_m5_list, gamma = snr.calcMagError_m5(magList, total_i_bandpass, m5, photParams)

In [ ]:
print 'maximum deviation in magnitude error: ',np.abs(mag_error_sed_list - mag_error_m5_list).max()

print 'value of magnitude at maximum deviation: ', \
       magList[np.argmax(np.abs(mag_error_sed_list - mag_error_m5_list))]